# 2021/1/11（鈴木・第５回）再帰関数と汎用的な全探索(2)

今日は、再帰関数を利用した汎用的な全探索（バックトラック）について学ぶ。

バックトラックでは、状態を１つずつ先に進めながら、有効な盤面であるかをチェックして、だめなら１つもどす、といった動きをする。
「深さ優先探索」の一種でもある。
たとえば、パズルの解を全探索するときにバックトラックが使える。

## itertools.product() と同様の全探索

ここでは、各要素が $0$ or $1$ の、長さ $n$ のリストをすべて列挙する関数 $f$ を再帰で記述する。

$f(n)$ は $f(n-1)$ の結果を使って以下のように書ける
- 先頭が $0$ の場合：残る成分を列挙するとちょうど $f(n-1)$
- 先頭が $1$ の場合：残る成分を列挙するとちょうど $f(n-1)$

なのでたとえば以下のコードは正しく動く。


In [ ]:
def f(n):
    if n == 0:
        return [[]]
    A0 = [[0]+res for res in f(n-1)]
    A1 = [[1]+res for res in f(n-1)]
    return A0 + A1

f(4)

しかしこのコードは、毎回リストの要素をコピーするので、非効率である。
そのかわりに、「グローバル変数」`res` を用意しておいて、
再帰関数を呼び出す過程で `res` の末尾に元を追加したり削除したりする。
すると効率のよいコードになる。言葉で言うと以下のような動作原理になっている。

- 「先頭を 0 に固定」に対応して、res に $0$ を追加
    - $n-1$ の場合を呼び出す（再帰呼び出し）
- 「先頭を 0 に固定」が終わったので、$0$ を削除
- 「先頭を 1 に固定」に対応して、res に $1$ を追加
    - $n-1$ の場合を呼び出す（再帰呼び出し）
- 「先頭を 1 に固定」が終わったので、$1$ を削除

再帰呼び出しの過程が木構造になっていることに注意してほしい。


In [1]:
# 未確定の長さが i
def f(i):
    if i == 0:
        print(res)
        return
    for k in range(2):
        res.append(k)
        f(i-1)
        res.pop()
    
res = [] # グローバル変数
f(4)


[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[0, 0, 1, 1]
[0, 1, 0, 0]
[0, 1, 0, 1]
[0, 1, 1, 0]
[0, 1, 1, 1]
[1, 0, 0, 0]
[1, 0, 0, 1]
[1, 0, 1, 0]
[1, 0, 1, 1]
[1, 1, 0, 0]
[1, 1, 0, 1]
[1, 1, 1, 0]
[1, 1, 1, 1]


個数のみを求めたいときは、例えば以下のように書く。

関数の外側で定義した値 `cnt` の中身を関数の中で直接変更したい場合は、
`global cnt` と書く。
すると関数内の変数 `cnt` と関数の外側の `cnt` を同じものと扱ってくれる。

`global` の宣言がなくても関数の外側のリストは変更することができる（リストはミュータブルだから）。
よってリスト `res` については `global` の宣言はいらない。


In [ ]:
# 未確定の長さが i
def f(i):
    global cnt # 「数」のグローバル変数は、このように記述すると変更が可能
    if i == 0:
        cnt += 1
        return
    for k in range(2):
        res.append(k)
        f(i-1)
        res.pop()
    
res = [] # グローバル変数（リスト）
cnt = 0  # グローバル変数（数）
f(4)
print(cnt)


ちなみにグローバル変数の多用はよくないとされている。（とくに複数人で大きなプログラムを書くとき）いつのまにか変数の中身が変更されてしまって、デバッグもできない、ということが起こりうる。

グローバル変数を使わない書き方もある。リスト `res` を変数として持ちまわってしまえばよい。そして個数 `cnt` については、再帰関数の帰り値としてしまう。

In [2]:
# 未確定の長さが i, 現在できている列が res
def f(i,res):
    if i == 0:
        return 1
    cnt = 0
    for k in range(2):
        res.append(k)
        cnt += f(i-1,res)
        res.pop()
    return cnt

cnt = f(16,[])
print(cnt)

65536


## 例題（枝刈り全探索）

$0,1,2$ を要素にもつ長さ $N$ のリストで、隣り合う数の差の絶対値がすべて $1$ となるようなものをすべて列挙する関数 $\mathrm{solve}(N)$ を作成せよ。


In [3]:
def solve(n):
    def check(res): # 列が条件をみたすなら True
        for i in range(n-1):
            if abs(res[i] - res[i+1]) != 1:
                return False
        return True

    def f(n):
        if n == 0:
            if check(res):
                print(res)
            return
        for i in range(3):
            res.append(i)
            f(n-1)
            res.pop()

    res = []
    f(n)

solve(4)

[0, 1, 0, 1]
[0, 1, 2, 1]
[1, 0, 1, 0]
[1, 0, 1, 2]
[1, 2, 1, 0]
[1, 2, 1, 2]
[2, 1, 0, 1]
[2, 1, 2, 1]


上のコードでは「関数内関数」を使った。

さて、上のコードでは、$3^N$ 通りの列をすべて作り、それぞれについて条件をチェックしていた。

そうではなく、全探索の途中で条件に合わなくなったらその先を調べないという効率化を **枝刈り** という。

下のコードでは、`if res == [] or abs(res[-1] - i) == 1` という部分で枝刈りをしている。
要素を追加するときに条件をチェックして、満たさないものはその先は調べない。




In [4]:
def f(n):
    if n == 0:
        print(res)
        return
    for i in range(3):
        if res == [] or abs(res[-1] - i) == 1:
            res.append(i)
            f(n-1)
            res.pop()

res = []
f(4)

[0, 1, 0, 1]
[0, 1, 2, 1]
[1, 0, 1, 0]
[1, 0, 1, 2]
[1, 2, 1, 0]
[1, 2, 1, 2]
[2, 1, 0, 1]
[2, 1, 2, 1]


## 例題：itertools.combinations() と同様の全探索

$0$ 以上 $N-1$ 以下の整数（$N$ 個）から $K$ 個を選び小さい順に並べる方法は $_NC_K$ 個ある。これらをすべて列挙せよ。




In [5]:
# 最大値 n, 残り個数 k (実際のところ、n は不変である)
def f(n,k):
    if k == 0: # K 個選んだ
        print(res)
        return
    if res == []:
        min_value = 0
    else:
        min_value = res[-1] + 1
    for i in range(min_value,n-k+1): # 右端を n-k+1 にしたのは枝刈り
        res.append(i)
        f(n,k-1)
        res.pop()

res = []
f(5,3)

[0, 1, 2]
[0, 1, 3]
[0, 1, 4]
[0, 2, 3]
[0, 2, 4]
[0, 3, 4]
[1, 2, 3]
[1, 2, 4]
[1, 3, 4]
[2, 3, 4]


## 例題：4*4 数独の解の探索

皆さんは数独というパズルをご存じだろうか。
普通数独は $9 \times 9$ のマスだが、ここでは$4 \times 4$ の数独を説明する。

$4 \times 4$のマスに $1$ から $4$ までの数のどれかを入れる。
ただし、同じ行、同じ列、盤面を４分割した $2 \times 2$ の箱の中には同じ数字が入ってはいけない、というルールである。

$4 \times 4$ の数独の盤面が２次元リストとして与えられる。ただし空白マスには 0 という数が入っている。数独の解をすべて列挙せよ。

（なお、以下の解は枝刈りの全くなされていない完全な全探索である。枝刈りをすればより効率的なコードとなる）


In [ ]:
# 行が異なる
def row_check():
    for i in range(4):
        r = [0,0,0,0]
        for j in range(4):
            v = board[i][j]-1
            if r[v]:
                return False
            r[v] = 1
    return True

# 列が異なる
def col_check():
    for j in range(4):
        r = [0,0,0,0]
        for i in range(4):
            v = board[i][j]-1
            if r[v]:
                return False
            r[v] = 1
    return True

# 2*2 が異なる。
def box_check():
    for dx,dy in [(0,0),(2,0),(0,2),(2,2)]:
        r = [0,0,0,0]
        for px,py in [(0,0),(0,1),(1,0),(1,1)]:
            v = board[px+dx][py+dy]-1
            if r[v]:
                return False
            r[v] += 1
    return True

# 整数 K に対応する座標 (i,j)
def num_to_pos(K):
    return K//4, K%4

# 盤面を見やすい形に表示
def print_board():
    for row in board:
        s = "".join(map(str,row))
        print(s)
    print()

# 4*4 の升目に、0~15 の番号をつける
# 今、整数 K に対応する位置を見ている
def solve(K):
    if K == 16: # 全部見た
        if row_check() and col_check() and box_check(): # 正しい盤面なら解をプリント
            print_board()
        return

    i,j = num_to_pos(K)
    # すでに数が埋まっていたら、次のマスへ進む
    if board[i][j] != 0:
        solve(K+1)
    # まだ埋まっていなかったら全パターンを考え、次のマスへ進む
    else: 
        for v in range(1,5):
            board[i][j] = v
            solve(K+1)
            board[i][j] = 0

board = [[1,2,3,4],[3,4,1,2],[4,0,0,0],[0,0,0,0]] # 初期盤面
solve(0)


## 練習問題

各要素が $0$ or $1$ or $2$ の、長さ $n$ のリストをすべて列挙する関数 $f$ を再帰で記述せよ。練習のため、`itertools.product()` は使わないこと。

$f(3)$ を実行して、27個のリストが出力されることを確かめよ。

In [ ]:
# 解答欄










**解答例**

In [ ]:
# 未確定の長さが i
def f(i):
    if i == 0:
        print(res)
        return
    for k in range(3):
        res.append(k)
        f(i-1)
        res.pop()
    
res = []
f(3)


[0, 0, 0]
[0, 0, 1]
[0, 0, 2]
[0, 1, 0]
[0, 1, 1]
[0, 1, 2]
[0, 2, 0]
[0, 2, 1]
[0, 2, 2]
[1, 0, 0]
[1, 0, 1]
[1, 0, 2]
[1, 1, 0]
[1, 1, 1]
[1, 1, 2]
[1, 2, 0]
[1, 2, 1]
[1, 2, 2]
[2, 0, 0]
[2, 0, 1]
[2, 0, 2]
[2, 1, 0]
[2, 1, 1]
[2, 1, 2]
[2, 2, 0]
[2, 2, 1]
[2, 2, 2]


## 練習問題

要素 $0,1$ からなる長さ $N$ のリストのうち、$1$ が隣り合わないものをすべて列挙する関数 $f(N)$ を作ろう。

たとえば $f(5)$ では $13$ 個のリストが出力される。確認してみよう。

In [ ]:
# 解答欄









**解答例**

In [ ]:
res = []
def f(N):
    if N == 0:
        print(res)
        return
    
    for i in range(2):
        if res == [] or i == 0 or res[-1] == 0:
            res.append(i)
            f(N-1)
            res.pop()

f(5)

[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 1, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 1]
[0, 1, 0, 0, 0]
[0, 1, 0, 0, 1]
[0, 1, 0, 1, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 1]
[1, 0, 0, 1, 0]
[1, 0, 1, 0, 0]
[1, 0, 1, 0, 1]


## 練習問題

$1$ 以上 $K$ 以下の整数を要素に持つ長さ $N$ のリストのうち、各要素が広義単調増加となるものは何通りあるか？

この問題の答えは $ {}_{N+K-1} C_{K-1}$ であるが、再帰で全探索することにより求めよ。
とくに $N = K= 7$ のとき答えが $1716$ となることを確かめよ。


In [ ]:
# 解答欄












**解答例** 

In [ ]:
# 未確定の長さ n, 最大値 k
def f(n,k,res):
    if n == 0:
        return 1
    # 「次に置けるのは v 以上の数」となる v を求める
    if res == []:
        v = 1
    else:
        v = res[-1]
    # v 以上 k 以下の数を全探索して再帰
    cnt = 0
    for i in range(v,k+1):
        res.append(i)
        cnt += f(n-1,k,res)
        res.pop()
    return cnt

f(7,7,[])

1716

## 練習問題（難しいかも）

$N \times N$ のグリッドの左上のマスに将棋の王（チェスのキング）の駒が置いてある（周囲８方向に移動できる）。
駒をちょうど $N^2-1$ 回移動させて、すべてのマスをちょうど１回ずつ踏む方法は何通りあるか？

$f(2) = 6$, $f(3) = 138$ を確かめよ。
下の解答例では $f(4)$ は$10$秒以内に求まる。皆さんのプログラムではどうか、試してみよう。


In [ ]:
# 解答欄








**解答例**

すでに移動したマスを二次元のリストで管理しながらバックトラックを行う。
再帰の引数として、現在のマスと残り移動回数を使う。

In [ ]:
# board[i][j]: すでに踏んだなら1, まだなら 0
# (i,j): いまいるマス
# num: 残り移動回数

def f(board,i,j,num):
    if num == 0:
        return 1
    cnt = 0
    # 周囲８マスを全探索
    for dx in range(-1,2):
        for dy in range(-1,2):
            # 移動しないのはダメ
            if dx == dy == 0:
                continue
            new_i = i + dx
            new_j = j + dy
            # 範囲外はダメ
            if new_i < 0 or new_j < 0 or new_i >= N or new_j >= N:
                continue
            # すでに踏んでいたらダメ
            if board[new_i][new_j] == 1:
                continue
            board[new_i][new_j] = 1
            cnt += f(board,new_i,new_j,num-1)
            board[new_i][new_j] = 0
    return cnt

N = 3
board = [[0]*N for _ in range(N)]
board[0][0] = 1
print(f(board,0,0,N*N-1))

138


## 練習問題（難しいかも）

N 以下の整数のうち、
$N = 2^a 3^b 5^c \cdots$ と表したとき、$a \ge b \ge c \ge \cdots$ となるものは何個あるか？

たとえば $3 = 2^0 \times 3^1$ は条件をみたさない。 
$1$ や $12 = 2^2 \times 3$ や、 $210 = 2 \times 3 \times 5 \times 7$ は条件をみたす。

たとえば $f(10) = 5$ である（$1,2,4,6,8$ が条件をみたす）。
また $f(100) = 16$ である。

とくに $f(10^9)$ を求めよ。

**ヒント**

素因数分解したときの素数としては `[2,3,5,7,11,13,17,19,23,29]` までを考えれば十分である（これらの積は $10^9$ を超えるので）。


In [ ]:
# 解答欄










**解答例**


必ずしもバックトラック（状態を戻す）を使う必要はなく、状態を引数で持つことで再帰すれば十分である。

In [ ]:
# これから i 番目の素数をチェックする
# これまでの積は v
# i-1 番目の素数を prev_num 回使った、
# 許される最大値は N
def f(i,v,prev_num,N):
    # prev_num = 0 なら、値は v で確定する
    if prev_num == 0:
        return 1
    # cnt: 答えの数
    # cur_num: 素数 p を何回使ったか
    cnt = cur_num = 0
    p = primes[i]
    # v, v*p, v*p^2, v*p^3, ... で条件をみたすものをすべて列挙 
    while v <= N and cur_num <= prev_num:
        cnt += f(i+1,v,cur_num,N)
        v *= p
        cur_num += 1
    return cnt

primes = [2,3,5,7,11,13,17,19,23,29]
print(f(0,1,100,10**9))


1274


## 練習問題（発展問題以上に難しい）

**この問題は解かずに、先に課題に目を通してください。**

$9 \times 9$ の数独のソルバーを実装せよ。
たとえば以下の問題が１分以内に解ける程度に高速なことが望ましい。

https://projecteuler.net/problem=96

**（時間に余裕がある人のみ挑戦してください。解答例は付けられません）**